In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv('data.csv')
cols=list(data.columns)

spends = [s for s in cols if s.endswith('Spend')]
impression= [s for s in cols if s.endswith('Impressions')]
CTR = [s for s in cols if s.endswith('CTR')]
CVRS = [s for s in cols if s.endswith('CVR (S)')]
CVRBF = [s for s in cols if s.endswith('CVR (BF)')]
CVRSV = [s for s in cols if s.endswith('CVR (SV)')]

sub=cols[0:18]

In [ ]:

def p2f(x):
  #change percentage to float
  if type(x) == str:
    return float(x.strip('%'))
  else:
    return x

# compute KPI mean value
kpi_means=[]
for kpi in [CTR,CVRS,CVRBF,CVRSV]:
  df=data[kpi]
  df=df.applymap(p2f)
  mean_KPI=df.mean(axis=1,skipna=True)
  kpi_means.append(mean_KPI)


#spend columns clean
def dollar2f(x):
  #change dollar string to float
  if type(x) == str:
    x=x.replace(',', '')
    return float(x.strip('$'))
  else:
    return x

spend_df=data[spends]
spend_df=spend_df.applymap(dollar2f)
mean_spend=spend_df.mean(axis=1,skipna=True)

In [ ]:
new=data[sub]

#variable descriptive statistics
new.describe(include='all')

,Creative ID,channel,funnel,publisher,lob,product,theme,kpi_audience,creative_versions,price,price_placement,discount,offer_placement,offer_group,asset_type,video_type,length,size
count,13271,13271,13271,13271,13271,13271,13271,13271,13271,13271,2464,10034.000000,2464,13271,13271,13179,13271,10814
unique,13271,2,3,18,2,7,14,16,14,8,5,NaN,7,7,9,6,7,12
top,Creative 1,Display,UF,Publisher 2,LOB 1,Product 3,Theme 2,Audience 1,Creative 3,No Price,Creative Only,NaN,Creative Only,No offer,Standard Banners,No Video Type,No Length,300x250
freq,1,10819,5512,7787,7915,5719,5139,5164,3999,6045,1147,NaN,1108,12009,6488,10718,10853,2298
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.981862,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.833948,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# chanve category variables to numeric variables
from sklearn.preprocessing import LabelEncoder 

#label encoder
labelencoder= LabelEncoder()

categoricals=list(new.columns)
unwanted_col = {'Creative ID', 'discount'}
categoricals = [ele for ele in categoricals if ele not in unwanted_col]

for i in categoricals:
  new[i] = labelencoder.fit_transform(new[i])

In [ ]:
#add kpi related columns
new['spend']=mean_spend
names=['CTR','CVRS','CVRBF','CVRSV']

#generate new data set
for i in range(len(names)):
  new[names[i]]=kpi_means[i]
new.head()

,Creative ID,channel,funnel,publisher,lob,product,theme,kpi_audience,creative_versions,price,price_placement,discount,offer_placement,offer_group,asset_type,video_type,length,size,spend,CTR,CVRS,CVRBF,CVRSV
0,Creative 1,1,1,0,0,1,0,0,0,0,5,0.0,7,0,8,4,4,12,318.769231,0.542308,0.0,0.000000,0.000000
1,Creative 2,0,1,10,0,1,0,8,0,0,5,0.0,7,0,1,2,6,1,2.714286,0.300000,0.0,0.000000,0.242857
2,Creative 3,0,1,10,0,1,0,8,0,0,5,0.0,7,0,1,2,6,3,13.142857,0.510000,0.0,15.138571,0.271429
3,Creative 4,0,1,10,0,1,0,8,0,0,5,0.0,7,0,1,2,6,4,1.166667,1.366667,0.0,0.000000,0.276667
4,Creative 5,0,1,10,0,1,0,8,0,0,5,0.0,7,0,1,2,6,5,12.142857,0.245714,0.0,0.000000,0.237143


In [ ]:
# output cleaned dataset
new.to_csv('clean_data.csv')